## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
from pprint import pprint

import gensim
import nltk
import numpy as np
import pandas as pd
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

nltk.download('wordnet')
np.random.seed(400)

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array
[nltk_data] Downloading package wordnet to /Users/dsp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [3]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [4]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [5]:
# past tense to present tense
print(WordNetLemmatizer().lemmatize('went', pos='v')) 

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [6]:
stemmer = SnowballStemmer('english')
original_words = [
    'caresses', 'flies', 'dies', 'mules', 'denied', 'died', 'agreed', 
    'owned', 'humbled', 'sized', 'meeting', 'stating', 'siezing', 
    'itemization','sensational', 'traditional', 'reference', 'colonizer',
    'plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'original word': original_words, 'stemmed': singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [7]:
'''
Write a function to perform the preprocessing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [8]:
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if (token not in gensim.parsing.preprocessing.STOPWORDS 
            and len(token) > 3):
            # TODO: Apply lemmatize_stemming() on the token, then add to 
            # the results list
            result.append(lemmatize_stemming(token))
    return result

In [9]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]
print('Original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [10]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes (it take 6 minutes on my laptop)

In [11]:
# TODO: preprocess all the headlines, saving the list of results as 
# 'processed_docs'
processed_docs = documents.headline_text.apply(preprocess)

In [12]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [13]:
'''
Create a dictionary from 'processed_docs' containing the number of times a
word appears in the training set using gensim.corpora.Dictionary and call 
it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [14]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.items():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [15]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned 
# above
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [16]:
'''
Create the Bag-of-words model for each document i.e for each document we 
create a dictionary reporting how many words and how many times those words
appear. Save this to 'bow_corpus'
'''
# TODO
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [17]:
'''
Checking Bag of Words corpus for our sample document --> 
(token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [18]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(
        bow_doc_4310[i][0], 
        dictionary[bow_doc_4310[i][0]], 
        bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [19]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and 
save it to 'tfidf'
'''

# TODO
tfidf = models.TfidfModel(bow_corpus)

In [20]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tfidf[bow_corpus]

In [21]:
'''
Preview TF-IDF scores for our first document --> (token_id, tfidf score)
'''
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.50554610985785686)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [22]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on
# your machine
# lda_model = gensim.models.LdaModel(
#     bow_corpus, num_topics=10, id2word=dictionary, passes=50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 
'lda_model'
'''
# TODO
lda_model = gensim.models.LdaModel(
    bow_corpus, num_topics=10, id2word=dictionary, passes=50, workers=4)

In [23]:
'''
For each topic, we will explore the words occuring in that topic and its 
relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.024*"test" + 0.017*"south" + 0.014*"inquiri" + 0.014*"law" + 0.013*"crew" + 0.013*"bushfir" + 0.013*"hear" + 0.013*"studi" + 0.012*"research" + 0.011*"suspect" 
Words: 0


Topic: 0.037*"govt" + 0.034*"council" + 0.034*"plan" + 0.033*"urg" + 0.030*"water" + 0.021*"fund" + 0.016*"seek" + 0.015*"group" + 0.015*"push" + 0.014*"help" 
Words: 1


Topic: 0.036*"kill" + 0.026*"iraq" + 0.017*"attack" + 0.016*"howard" + 0.016*"arrest" + 0.016*"talk" + 0.015*"break" + 0.015*"protest" + 0.015*"troop" + 0.014*"nuclear" 
Words: 2


Topic: 0.033*"warn" + 0.023*"miss" + 0.021*"hous" + 0.019*"polic" + 0.016*"search" + 0.015*"rise" + 0.015*"industri" + 0.015*"claim" + 0.014*"centr" + 0.013*"water" 
Words: 3


Topic: 0.024*"chang" + 0.021*"labor" + 0.020*"opposit" + 0.018*"elect" + 0.017*"concern" + 0.017*"green" + 0.016*"price" + 0.016*"call" + 0.015*"govt" + 0.014*"back" 
Words: 4


Topic: 0.064*"polic" + 0.035*"charg" + 0.032*"crash" + 0.030*"court" + 0.028*"face" + 0.023*"investig" + 0.019*"

## Step 4.2 Running LDA using TF-IDF ##

In [24]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(
    corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [25]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.006*"uranium" + 0.006*"news" + 0.005*"market" + 0.005*"govt" + 0.005*"kangaroo" + 0.005*"refus" + 0.005*"ash" + 0.005*"polic" + 0.004*"suspend" + 0.004*"concern"


Topic: 1 Word: 0.009*"govt" + 0.007*"hick" + 0.006*"action" + 0.006*"urg" + 0.006*"council" + 0.005*"teacher" + 0.005*"plan" + 0.005*"nurs" + 0.005*"prison" + 0.005*"bird"


Topic: 2 Word: 0.021*"closer" + 0.014*"water" + 0.008*"fund" + 0.008*"govt" + 0.007*"boost" + 0.007*"plan" + 0.006*"council" + 0.006*"urg" + 0.006*"price" + 0.006*"drought"


Topic: 3 Word: 0.024*"crash" + 0.014*"polic" + 0.012*"investig" + 0.009*"fatal" + 0.009*"die" + 0.007*"highway" + 0.006*"plane" + 0.006*"death" + 0.006*"victim" + 0.005*"woman"


Topic: 4 Word: 0.018*"miss" + 0.016*"search" + 0.008*"iraq" + 0.008*"bodi" + 0.007*"solomon" + 0.007*"polic" + 0.006*"troop" + 0.005*"afghanistan" + 0.005*"continu" + 0.005*"qanta"


Topic: 5 Word: 0.018*"kill" + 0.010*"bomb" + 0.007*"blast" + 0.007*"attack" + 0.007*"israel" + 0.006*"iraq" 

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [26]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.322396457195282	 
Topic: 0.036*"kill" + 0.026*"iraq" + 0.017*"attack" + 0.016*"howard" + 0.016*"arrest" + 0.016*"talk" + 0.015*"break" + 0.015*"protest" + 0.015*"troop" + 0.014*"nuclear"

Score: 0.31759676337242126	 
Topic: 0.037*"govt" + 0.034*"council" + 0.034*"plan" + 0.033*"urg" + 0.030*"water" + 0.021*"fund" + 0.016*"seek" + 0.015*"group" + 0.015*"push" + 0.014*"help"

Score: 0.22000005841255188	 
Topic: 0.024*"test" + 0.017*"south" + 0.014*"inquiri" + 0.014*"law" + 0.013*"crew" + 0.013*"bushfir" + 0.013*"hear" + 0.013*"studi" + 0.012*"research" + 0.011*"suspect"

Score: 0.02000400982797146	 
Topic: 0.026*"sydney" + 0.023*"power" + 0.017*"win" + 0.014*"lose" + 0.014*"final" + 0.012*"forc" + 0.012*"sale" + 0.012*"compani" + 0.012*"fall" + 0.011*"start"

Score: 0.02000139281153679	 
Topic: 0.064*"polic" + 0.035*"charg" + 0.032*"crash" + 0.030*"court" + 0.028*"face" + 0.023*"investig" + 0.019*"accus" + 0.018*"murder" + 0.017*"drug" + 0.014*"woman"

Score: 0.0200013704597949

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [28]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5841127038002014	 
Topic: 0.021*"closer" + 0.014*"water" + 0.008*"fund" + 0.008*"govt" + 0.007*"boost" + 0.007*"plan" + 0.006*"council" + 0.006*"urg" + 0.006*"price" + 0.006*"drought"

Score: 0.255874902009964	 
Topic: 0.009*"govt" + 0.007*"hick" + 0.006*"action" + 0.006*"urg" + 0.006*"council" + 0.005*"teacher" + 0.005*"plan" + 0.005*"nurs" + 0.005*"prison" + 0.005*"bird"

Score: 0.020003439858555794	 
Topic: 0.009*"south" + 0.008*"east" + 0.008*"firefight" + 0.008*"timor" + 0.007*"nuclear" + 0.006*"australia" + 0.005*"korea" + 0.005*"test" + 0.005*"india" + 0.005*"north"

Score: 0.020002607256174088	 
Topic: 0.024*"crash" + 0.014*"polic" + 0.012*"investig" + 0.009*"fatal" + 0.009*"die" + 0.007*"highway" + 0.006*"plane" + 0.006*"death" + 0.006*"victim" + 0.005*"woman"

Score: 0.02000134252011776	 
Topic: 0.018*"miss" + 0.016*"search" + 0.008*"iraq" + 0.008*"bodi" + 0.007*"solomon" + 0.007*"polic" + 0.006*"troop" + 0.005*"afghanistan" + 0.005*"continu" + 0.005*"qanta"

Score:

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [29]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.41999998688697815	 Topic: 0.024*"test" + 0.017*"south" + 0.014*"inquiri" + 0.014*"law" + 0.013*"crew"
Score: 0.41999998688697815	 Topic: 0.064*"polic" + 0.035*"charg" + 0.032*"crash" + 0.030*"court" + 0.028*"face"
Score: 0.019999999552965164	 Topic: 0.037*"govt" + 0.034*"council" + 0.034*"plan" + 0.033*"urg" + 0.030*"water"
Score: 0.019999999552965164	 Topic: 0.036*"kill" + 0.026*"iraq" + 0.017*"attack" + 0.016*"howard" + 0.016*"arrest"
Score: 0.019999999552965164	 Topic: 0.033*"warn" + 0.023*"miss" + 0.021*"hous" + 0.019*"polic" + 0.016*"search"
Score: 0.019999999552965164	 Topic: 0.024*"chang" + 0.021*"labor" + 0.020*"opposit" + 0.018*"elect" + 0.017*"concern"
Score: 0.019999999552965164	 Topic: 0.022*"health" + 0.021*"minist" + 0.019*"want" + 0.018*"union" + 0.016*"return"
Score: 0.019999999552965164	 Topic: 0.025*"open" + 0.022*"road" + 0.022*"lead" + 0.021*"australia" + 0.020*"jail"
Score: 0.019999999552965164	 Topic: 0.026*"closer" + 0.021*"defend" + 0.021*"year" + 0.021

The model correctly classifies the unseen document with 'x'% probability to the X category.